<a href="https://colab.research.google.com/github/deacs11/Economic-Policy-Simulator/blob/main/Economic_Policy_Simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup and Initialization

In [1]:
!pip install -q pyautogen openai python-dotenv fpdf2

from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from google.colab import userdata
import os

# 🔐 Load API key
api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ No API key found! Check runtime secrets.")
print("✅ API key loaded.")

llm_config = {
    "config_list": [{
        "model": "gpt-4",
        "api_key": api_key
    }]
}


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.7/245.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.5 MB/s eta 0:00:00
✅ API key loaded.


Define Economic Agents

In [2]:
keynesian_minister = AssistantAgent(
    name="KeynesianMinister",
    system_message="You are a Keynesian economist. Propose fiscal stimulus and government intervention.",
    llm_config=llm_config
)

central_banker = AssistantAgent(
    name="CentralBankGovernor",
    system_message="You are a monetarist. Focus on controlling inflation and adjusting interest rates.",
    llm_config=llm_config
)

business_representative = AssistantAgent(
    name="BusinessRep",
    system_message="You represent the voice of business owners. Advocate for market-friendly and pro-growth policies.",
    llm_config=llm_config
)

union_leader = AssistantAgent(
    name="UnionLeader",
    system_message="You speak for the working class. Defend employment, wages, and labor rights.",
    llm_config=llm_config
)

independent_office = AssistantAgent(
    name="EconomicOffice",
    system_message="You are an independent analyst. Provide neutral impact assessments and projections.",
    llm_config=llm_config
)

User Agent

In [3]:
moderator = UserProxyAgent(
    name="HumanModerator",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": ".", "use_docker": False}
)


Assemble the Virtual Ministry

In [4]:
economic_team = [
    keynesian_minister,
    central_banker,
    business_representative,
    union_leader,
    independent_office,
    moderator
]

econ_chat = GroupChat(
    agents=economic_team,
    messages=[],
    max_round=5
)

econ_manager = GroupChatManager(
    groupchat=econ_chat,
    llm_config=llm_config
)

Define Scenario and Trigger Debate

In [5]:
crisis_prompt = """
The economy is entering a stagflation scenario: inflation is high, GDP is contracting, and unemployment is rising.

As a virtual economic policy team, propose and discuss strategic responses from your perspectives. Evaluate pros and cons.
"""

moderator.initiate_chat(
    econ_manager,
    message=crisis_prompt
)

HumanModerator (to chat_manager):


The economy is entering a stagflation scenario: inflation is high, GDP is contracting, and unemployment is rising.

As a virtual economic policy team, propose and discuss strategic responses from your perspectives. Evaluate pros and cons.


--------------------------------------------------------------------------------

Next speaker: KeynesianMinister

KeynesianMinister (to chat_manager):

In response to this economic crisis, from a Keynesian perspective, we should use fiscal stimulus measures and active government intervention to stimulate economic activity and employment. We can primarily do this through two mechanisms: increase government spending and enact tax cuts. 

First, the government could increase direct spending on infrastructure projects, public works, education, and healthcare. This could lead to job creation and an increase in aggregate demand, which can spur an economic recovery. This government-driven increase in demand would employ

ChatResult(chat_id=None, chat_history=[{'content': '\nThe economy is entering a stagflation scenario: inflation is high, GDP is contracting, and unemployment is rising.\n\nAs a virtual economic policy team, propose and discuss strategic responses from your perspectives. Evaluate pros and cons.\n', 'role': 'assistant', 'name': 'HumanModerator'}, {'content': "In response to this economic crisis, from a Keynesian perspective, we should use fiscal stimulus measures and active government intervention to stimulate economic activity and employment. We can primarily do this through two mechanisms: increase government spending and enact tax cuts. \n\nFirst, the government could increase direct spending on infrastructure projects, public works, education, and healthcare. This could lead to job creation and an increase in aggregate demand, which can spur an economic recovery. This government-driven increase in demand would employ idle resources, producing goods and services, and the increased emp

Synthesize the Discussion

In [7]:
synthesizer = AssistantAgent(
    name="Synthesizer",
    system_message="Summarize the debate with a balanced executive summary and highlight key proposals.",
    llm_config=llm_config
)

full_discussion = "\n\n".join([
    f"{msg['role']}: {msg['content']}" for msg in econ_chat.messages
])

response = synthesizer.generate_reply(messages=[
    {"role": "user", "content": f"Here is the discussion transcript:\n\n{full_discussion}\n\nSummarize the debate."}
])

summary = response

In [10]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=55bb27cb2a84799fbe3692f659c42d7fdba1a368b8ee60a125ebb42c6892ac03
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [12]:
import unicodedata

# 🔧 Remove all characters not supported by 'latin-1'
def normalize_to_ascii(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

# ✂️ Normalize the transcript and summary
clean_transcript = normalize_to_ascii(full_discussion)
clean_summary = normalize_to_ascii(summary)

Export to PDF

In [15]:
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()

pdf.set_font("Helvetica", "B", size=14)
pdf.cell(200, 10, "Economic Policy Simulation Output", ln=True, align="C")

pdf.ln(10)
pdf.set_font("Helvetica", "B", size=12)
pdf.cell(200, 10, "Debate Transcript", ln=True)

pdf.set_font("Helvetica", "", size=11)
pdf.multi_cell(0, 8, clean_transcript)

pdf.ln(10)
pdf.set_font("Helvetica", "B", size=12)
pdf.cell(200, 10, "AI-Generated Summary", ln=True)

pdf.set_font("Helvetica", "", size=11)
pdf.multi_cell(0, 8, clean_summary)

pdf.output("economic_policy_simulator.pdf")

''